In [1]:
!pip install -q ultralytics
!pip uninstall -y wandb

Found existing installation: wandb 0.16.5
Uninstalling wandb-0.16.5:
  Successfully uninstalled wandb-0.16.5


In [5]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image
from ultralytics import YOLO
from collections import defaultdict
from scipy.optimize import linear_sum_assignment

In [6]:
import os
# Path to the training images directory
train_images_dir = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
test_images_dir = '/kaggle/input/coco-2017-dataset/coco2017/test2017'
val_images_dir = '/kaggle/input/coco-2017-dataset/coco2017/val2017'

train_images_count = len(os.listdir(train_images_dir))
test_images_count = len(os.listdir(test_images_dir))
val_images_count = len(os.listdir(val_images_dir))
print(f"Number of images in the training set: {train_images_count}, test set: {test_images_count}, validation set: {val_images_count}")

Number of images in the training set: 118287, test set: 40670, validation set: 5000


In [7]:
import json
import os
from shutil import copy2
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

# Define the COCO dataset directory and the new directory for person class data
coco_dir = '/kaggle/input/coco-2017-dataset/coco2017'
yolov8_dir = '/kaggle/working/yolov8'

# Create the necessary directories
phases = ['train','val']  
for phase in phases:
    os.makedirs(os.path.join(yolov8_dir, phase, 'images'), exist_ok=True)
    os.makedirs(os.path.join(yolov8_dir, phase, 'labels'), exist_ok=True)

# Function to convert COCO bounding box to normalized YOLO format
def convert_to_yolo(box, img_width, img_height):
    x_center = (box[0] + box[2] / 2) / img_width
    y_center = (box[1] + box[3] / 2) / img_height
    width = box[2] / img_width
    height = box[3] / img_height
    return x_center, y_center, width, height

# Function to process each image and its annotations
def process_image(data):
    img, person_annotations, phase = data
    img_id = img['id']
    img_width, img_height = img['width'], img['height']
    file_name = img['file_name']
    output_dir = os.path.join(yolov8_dir, phase)  # Adjusted to use phase-specific folders

    # Prepare label data
    label_data = []
    for ann in person_annotations:
        if ann['image_id'] == img_id:
            bbox = convert_to_yolo(ann['bbox'], img_width, img_height)
            label_data.append(f"0 {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}")

    # Write label file
    label_file_path = os.path.join(output_dir, 'labels', f'{os.path.splitext(file_name)[0]}.txt')
    with open(label_file_path, 'w') as label_file:
        label_file.write('\n'.join(label_data))

    # Copy the image
    src_path = os.path.join(coco_dir, f'{phase}2017', file_name)
    dst_path = os.path.join(output_dir, 'images', file_name)
    copy2(src_path, dst_path)

for phase in phases:

    annotations_file = f'instances_{phase}2017.json'
    annotations_path = os.path.join(coco_dir, 'annotations', annotations_file)

    with open(annotations_path) as f:
        annotations_data = json.load(f)

    print(f"Processing {phase} annotations...")
    person_category_id = next(cat['id'] for cat in annotations_data['categories'] if cat['name'] == "person")

    # Create a mapping from image_id to annotations
    annotations_by_image_id = {}
    for annotation in annotations_data['annotations']:
        if annotation['category_id'] == person_category_id:
            if annotation['image_id'] not in annotations_by_image_id:
                annotations_by_image_id[annotation['image_id']] = []
            annotations_by_image_id[annotation['image_id']].append(annotation)

    # Now use the mapping to create img_data_list efficiently
    img_data_list = [(img, annotations_by_image_id.get(img['id'], []), phase) 
                     for img in annotations_data['images'] 
                     if img['id'] in annotations_by_image_id]

    with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
        list(tqdm(executor.map(process_image, img_data_list), total=len(img_data_list), desc=f'Processing {phase} images'))


Processing train annotations...


Processing train images: 100%|██████████| 64115/64115 [03:44<00:00, 285.07it/s]


Processing val annotations...


Processing val images: 100%|██████████| 2693/2693 [00:08<00:00, 306.62it/s]


In [8]:
import os
import random

# Define source and destination directories
src_images_dir = '/kaggle/working/yolov8/train/images'
src_labels_dir = '/kaggle/working/yolov8/train/labels'
dest_images_dir = '/kaggle/working/yolov8/test/images'
dest_labels_dir = '/kaggle/working/yolov8/test/labels'

# Ensure destination directories exist
os.makedirs(dest_images_dir, exist_ok=True)
os.makedirs(dest_labels_dir, exist_ok=True)

# List all image files in the source directory
image_files = [f for f in os.listdir(src_images_dir) if os.path.isfile(os.path.join(src_images_dir, f))]

# Randomly select 10% of these image files
selected_images = random.sample(image_files, max(1, len(image_files) // 10))  # Ensure at least one file is selected if not empty

# Move selected image files and their corresponding label files to the destination directories
for image_file in tqdm(selected_images):
    # Move image
    src_image_path = os.path.join(src_images_dir, image_file)
    dest_image_path = os.path.join(dest_images_dir, image_file)
    os.rename(src_image_path, dest_image_path)
    
    label_file = os.path.splitext(image_file)[0] + '.txt'  # Assuming label files are .txt with same base filename
    src_label_path = os.path.join(src_labels_dir, label_file)
    dest_label_path = os.path.join(dest_labels_dir, label_file)
    if os.path.exists(src_label_path):  
        os.rename(src_label_path, dest_label_path)

print(f"Moved {len(selected_images)} images and their corresponding labels to test directories.")


100%|██████████| 6411/6411 [00:00<00:00, 13848.98it/s]

Moved 6411 images and their corresponding labels to test directories.


In [9]:
config = """
path: /kaggle/working/yolov8  
train: train                
val: test                      

names:
  0: person
 
"""
with open("/kaggle/working/coco.yaml", 'w') as f:
    f.write(config)

In [10]:
from ultralytics import YOLO
model = YOLO('yolov8s.pt')
data_yaml = '/kaggle/working/coco.yaml' 
results = model.train(data=data_yaml, epochs=2, imgsz=640,batch=128, classes=[0],device=[0,1])

100%|██████████| 21.5M/21.5M [00:00<00:00, 165MB/s]


Ultralytics YOLOv8.1.47 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/coco.yaml, epochs=2, time=None, patience=100, batch=128, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=[0], retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fals

100%|██████████| 755k/755k [00:00<00:00, 13.3MB/s]
2024-04-16 03:23:31,810	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-16 03:23:34,436	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-16 03:23:39.431320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 03:23:39.431423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 03:23:39.656259: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS facto

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

2024-04-16 03:24:01.971839: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 03:24:01.971900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 03:24:01.973555: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

                   all       6411      26473      0.774      0.656      0.742      0.496

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/2      14.5G      1.063     0.8647      1.146        358        640: 100%|██████████| 451/451 [12:14<00:00,  1.63s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:54<00:00,  1.08s/it]


                   all       6411      26473      0.786      0.669      0.757      0.519

2 epochs completed in 0.437 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:54<00:00,  1.07s/it]


                   all       6411      26473      0.786      0.669      0.756      0.519
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train


In [11]:
# function to calculate Intersection over Union (IoU) for two bounding boxes
def calc_iou(box1, box2): # box1 and box2 are (x, y, w, h)
    # Calculate the coordinates of the corners of the two boxes
    x1_min = box1[0] - box1[2] / 2 # x - w/2
    x1_max = box1[0] + box1[2] / 2 # x + w/2
    y1_min = box1[1] - box1[3] / 2 # y - h/2
    y1_max = box1[1] + box1[3] / 2 # y + h/2

    x2_min = box2[0] - box2[2] / 2 # x - w/2
    x2_max = box2[0] + box2[2] / 2 # x + w/2
    y2_min = box2[1] - box2[3] / 2 # y - h/2
    y2_max = box2[1] + box2[3] / 2 # y + h/2

    # Calculate the coordinates for the intersection rectangle
    inter_x_min = max(x1_min, x2_min) # max of the two x mins
    inter_y_min = max(y1_min, y2_min) # max of the two y mins
    inter_x_max = min(x1_max, x2_max) # min of the two x maxes
    inter_y_max = min(y1_max, y2_max) # min of the two y maxes 

    # If there is no intersection, return 0
    if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
        return 0.0

    # Calculate intersection area
    inter_area = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)

    # Calculate area of both bounding boxes
    area1 = (x1_max - x1_min) * (y1_max - y1_min)
    area2 = (x2_max - x2_min) * (y2_max - y2_min)

    # Compute the Intersection over Union by taking the intersection area and dividing it by the sum of prediction + ground-truth areas - the interesection area
    iou = inter_area / float(area1 + area2 - inter_area)

    # Ensure IOU is within correct bounds
    assert 0.0 <= iou <= 1.0, "IOU must be between 0 and 1"
    return iou

In [14]:
import os
import numpy as np
from collections import defaultdict
from scipy.optimize import linear_sum_assignment

yolov8_dir = "/kaggle/working/yolov8/"
test_images_folder = yolov8_dir + "test/images/"
test_labels_folder = yolov8_dir + "test/labels/"
test_files = os.listdir(test_images_folder)

sum_iou = 0
ious = defaultdict(float) # Store the IOU for each test image
num_tested = 0  # Count of images actually tested (i.e., having both gt and preds)

ground_truths = {} # Store the ground truths for each test image 
for test_file in test_files:
    label_file = test_labels_folder + os.path.splitext(test_file)[0] + '.txt' # Get the label file for the test image
    with open(label_file, 'r') as file:
        lines = file.readlines() # Read the lines of the label file
        ground_truths[test_file] = [list(map(float, line.strip().split())) for line in lines] # Store the ground truths

# Iterate through the test files to test the performance
for test_file in tqdm(test_files):
    res = model.predict(test_images_folder + test_file, classes=[0], verbose=False) # Get the predictions for the test image
    
    gt = np.array(ground_truths[test_file]) # Get the ground truths for the test image
    preds = res[0].boxes.xywhn.cpu().numpy() # Get the predictions for the test image
    
    if len(gt) == 0 or len(preds) == 0: # Skip the mean IOU calculation for this file if there are no gt or predictions
        continue  

    num_tested += 1  # Increment only if there are both gt and predictions

    iou_matrix = np.zeros((len(gt), len(preds))) # Initialize the IOU matrix
    for i in range(len(gt)): # Calculate the IOU for each pair of gt and prediction
        for j in range(len(preds)): 
            iou_matrix[i, j] = calc_iou(gt[i], preds[j])

    # Do the Hungarian matching algorithm
    gt_idx, pred_idx = linear_sum_assignment(1 - iou_matrix)
    assigned_ious = iou_matrix[gt_idx, pred_idx]
    
    # Compute mean across all instances in the image
    mean_iou = np.mean(assigned_ious) 

    assert mean_iou <= 1.0, "Mean IOU should not exceed 1.0"
    
    sum_iou += mean_iou
    ious[test_file] = (mean_iou, assigned_ious)

100%|██████████| 6411/6411 [01:56<00:00, 54.84it/s]


In [18]:
average_iou = sum_iou / num_tested if num_tested > 0 else 0
print(f"Average IOU: {average_iou}")

Average IOU: 0.7522543195211435
